In [1]:
!pip install pandas

  Using cached pandas-2.2.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (89 kB)
  Using cached tzdata-2024.2-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.2.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.7 MB)
Using cached tzdata-2024.2-py2.py3-none-any.whl (346 kB)


In [2]:
import os
import json
import pandas as pd

# 3sent 폴더가 포함된 상위 경로 설정
base_dir = r'/home/metaai2/byeonguk_work/I-live-A-commerce/TL1'  # 데이터가 저장된 상위 경로
categories = ['c_event', 'culture', 'enter', 'fm_drama', 'fs_drama', 'history']

data_list = []  # 본문 데이터를 저장할 리스트

# 각 카테고리의 3sent 폴더에서 데이터 추출
for category in categories:
    three_sent_dir = os.path.join(base_dir, category, '20per')  # 3sent 폴더 경로

    # 3sent 폴더의 모든 파일 처리
    for filename in os.listdir(three_sent_dir):
        file_path = os.path.join(three_sent_dir, filename)

        # JSON 파일 읽기
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)

            # 본문 데이터 추출
            text = data.get('Meta', {}).get('passage', '')

            # 본문이 존재하면 저장
            if text:
                data_list.append({'category': category, 'text': text})

# 데이터프레임으로 변환 및 저장
df = pd.DataFrame(data_list)
df.to_csv('20per_data.csv', index=False, encoding='utf-8')
print(f"총 데이터 개수: {len(df)}")


총 데이터 개수: 40200


In [3]:
!pip install torch transformers pandas tqdm


In [4]:
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from transformers import PreTrainedTokenizerFast

# KoBART 토크나이저 로드
tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-base-v2')

# 학습 데이터 로드
data = pd.read_csv('20per_data.csv')

# 데이터셋 클래스 정의
class KoBARTDataset(Dataset):
    def __init__(self, tokenizer, data, max_len=512):
        self.tokenizer = tokenizer
        self.data = data
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = self.data.loc[idx, 'text']  # 본문 데이터

        # 입력 텍스트와 출력 텍스트 생성
        inputs = self.tokenizer(
            text, max_length=self.max_len, truncation=True, return_tensors='pt', padding='max_length'
        )
        targets = self.tokenizer(
            text, max_length=self.max_len, truncation=True, return_tensors='pt', padding='max_length'
        )

        return {
            'input_ids': inputs['input_ids'].squeeze(),
            'attention_mask': inputs['attention_mask'].squeeze(),
            'labels': targets['input_ids'].squeeze()
        }

# 데이터셋 및 DataLoader 생성
dataset = KoBARTDataset(tokenizer, data)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)


/home/metaai2/anaconda3/envs/dongsub/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [5]:
from transformers import BartForConditionalGeneration, AdamW
from tqdm import tqdm

# KoBART 모델 로드
model = BartForConditionalGeneration.from_pretrained('gogamza/kobart-base-v2')

# 옵티마이저 설정
optimizer = AdamW(model.parameters(), lr=5e-5)

# 학습 설정
epochs = 3  # 학습 에포크 수


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
/home/metaai2/anaconda3/envs/dongsub/lib/python3.12/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [6]:
import torch
from tqdm import tqdm

# GPU 디바이스 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"사용 중인 디바이스: {device}")

# 모델과 데이터를 GPU로 전송
model = model.to(device)

for epoch in range(epochs):
    model.train()
    epoch_loss = 0

    for batch in tqdm(dataloader):
        # 배치 데이터를 GPU로 전송
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()

        # 모델 출력 및 손실 계산
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    print(f"Epoch {epoch + 1} Loss: {epoch_loss / len(dataloader)}")


사용 중인 디바이스: cuda


100%|██████████| 5025/5025 [11:06<00:00,  7.54it/s]


Epoch 1 Loss: 0.015812587825127063


100%|██████████| 5025/5025 [11:07<00:00,  7.53it/s]


Epoch 2 Loss: 0.0016382119948438552


100%|██████████| 5025/5025 [11:07<00:00,  7.53it/s]

Epoch 3 Loss: 0.0014195657006051653


In [7]:
model.save_pretrained('./kobart-finetuned-20per')
tokenizer.save_pretrained('./kobart-finetuned-20per')


/home/metaai2/anaconda3/envs/dongsub/lib/python3.12/site-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'forced_eos_token_id': 1}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


('./kobart-finetuned-20per/tokenizer_config.json',
 './kobart-finetuned-20per/special_tokens_map.json',
 './kobart-finetuned-20per/tokenizer.json')

In [8]:
def summarize(text, model, tokenizer):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    summary_ids = model.generate(inputs['input_ids'], max_length=150, min_length=40, length_penalty=2.0)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# 테스트 예시
test_text = "지난해 5월 숨진 '로켓배송' 기사 정슬기 씨.

쿠팡CLS 대리점과 근로계약이 아닌 위탁계약을 맺고 일했습니다.

과로사로 산재를 인정받은 정 씨는 이른바 '3회전' 배송 방식으로 하루 평균 279건을 배송했습니다.

새벽 배송을 마치기 위해 밤 9시부터 오전 7시까지 배송 지역을 매일 세 번이나 돈 겁니다.

[정금석/고 정슬기 씨 아버지/지난 14일 : "(배송)캠프하고 제 아들이 배송한 지역이 상봉동인데 상봉동을 그게 한 20km 된답니다. 거기를 세 번 왔다 갔다 하는 거예요."]

이런 업무 강도, 숨진 정 씨만의 일이 아닙니다.

KBS가 입수한 고용노동부의 설문조사 결과입니다.

응답한 대리점 배송기사 767명 가운데 76.8%, 네 명 중 세 명이 야간에 3회전 배송을 하고 있었습니다.

하루 250건 이상을 배송한다는 응답도 76.4%에 달했습니다.

쿠팡CLS에 직고용된 기사들은 3회전 배송 비율이 38%, 250건 이상 배송은 10%대인 것과 차이가 큽니다.

고강도 업무지만 노동 시간도 법정 근로 시간보다 길었습니다.

하루 평균 근무 시간은 9시간 26분, 일주일에 평균 5.5일 일했습니다.

특히 폭우, 폭설 등 악천후에도 "무조건 배송을 한다"는 응답이 전체의 77%나 됐습니다.

[김주영/국회 환경노동위원/더불어민주당 : "몸에 무리가 올 수밖에 없는 노동 환경이라는 것이 이번 실태조사를 통해 나타났습니다. 좀 더 세심한 근로 시간 규제가 필요하다고 (볼 수 있습니다)."]

퀵플렉서로 불리는 이들 특수고용직 배송기사들은 쿠팡에 간접 고용된 형태로 건별로 배송 수수료를 받습니다.

고용부는 최근 이들이 근로기준법상 근로자가 아니라면서도, 노동 환경 개선이 필요하다고 지적했습니다.

쿠팡 측은 이번 조사 결과에 대해 별도 입장을 내지 않았습니다.

21일 국회에선 '쿠팡 청문회'가 예정돼 있습니다.

KBS 뉴스 김채린입니다."
print("본문:", test_text)
print("요약 결과:", summarize(test_text, model, tokenizer))


SyntaxError: unterminated string literal (detected at line 7) (404459336.py, line 7)